In [52]:
# import sys
# !{sys.executable} -m pip install lightfm
import random
import itertools

import pickle

import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from collections import defaultdict

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# export SPOTIPY_CLIENT_ID='95ca7ded0e274316a1c21476f83e1576'
# export SPOTIPY_CLIENT_SECRET='15ee20c2e8924c80b5693dd9f26daa95'
# export SPOTIPY_REDIRECT_URI='your-app-redirect-url'
SPOTIPY_CLIENT_ID2='855879a1dbba413297f108ab660738ed'
SPOTIPY_CLIENT_SECRET2='f3bd56217f4d4b5b8c8b5898f41cd0be'

In [53]:
v = 1
p = 0.1

In [54]:
# df_playlist = pd.read_csv('./data/spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)
df_playlist = pd.read_csv('./data/spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and (i % v != 0))

In [55]:
df_playlist.shape

(12891680, 4)

In [56]:
df_playlist.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [57]:
df_playlist.columns = df_playlist.columns.str.replace('"', '')
df_playlist.columns = df_playlist.columns.str.replace('name', '')
df_playlist.columns = df_playlist.columns.str.replace(' ', '')
df_playlist.columns
df_playlist.head()

,user_id,artist,track,playlist
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [ ]:
## Adding custom users into interactions receives list of usernames
def addUsersToDataSet(df_playlist, users):
    auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID2, client_secret=SPOTIPY_CLIENT_SECRET2)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    new_user_rows = []
    for username in users:
        user = sp.user(username)
        user_id = user['id']
        display_name = get_display_name_from_username(username)
        playlists = sp.user_playlists(username)
        playlist_names = get_user_playlists_names(sp, username)
        playlist_ids = get_user_playlists_ids(sp, username)
        song_ids = []
        for playlist_id in playlist_ids:
            current_playlist_song_ids = get_song_ids_in_playlist(sp, playlist_id)
            song_ids.extend(current_playlist_song_ids)
        for song_id in song_ids:
            track = sp.track(song_id)

            name = track['name']
            artist = track['album']['artists'][0]['name']

            # Create a dictionary containing a song's name, artist, and song ID -> Add this dictionary to a list 
            new_row = {
                'user_id': display_name,
                'track': name,
                'artist': artist,
                'playlist': playlist_names[0]
            }
            new_user_rows.append(new_row)
        
#     for row in new_user_rows:
#         print(row)
    
    temp_df = pd.DataFrame(new_user_rows) 
    df_playlist = pd.concat([df_playlist, temp_df], ignore_index = True)
    
    return df_playlist

def get_user_playlists_names(sp, username):
    tmp = []
    playlists = sp.user_playlists(username)
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            tmp.append(playlist['name'])
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            playlists = None
    return tmp

def get_user_playlists_ids(sp, username):
    tmp = []
    playlists = sp.user_playlists(username)
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            tmp.append(playlist['uri'])
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            playlists = None
    return tmp

def get_song_ids_in_playlist(sp, playlist_id):
    playlist_tracks = []
    playlist_tracks_raw = sp.playlist_tracks(playlist_id=playlist_id, fields='items(track(id))')
    for track in playlist_tracks_raw['items']:
        playlist_tracks.append(track["track"]["id"])
    return playlist_tracks

# Display names will be used for getting recommendations
def get_display_name_from_username(username):
    if (username == '31jcprt274yhbqbldnen6q4r2rxq'):
        return 'billywalton98765'
    if (username == 'stevienash98765'):
        return 'stevienash98765'
    if (username == '31wexqymjgzdkkved4vcwote7r2q'):
        return 'plazadruben'
    
def get_username_from_display_name(displayname):
    if (username == 'billywalton98765'):
        return '31jcprt274yhbqbldnen6q4r2rxq'
    if (username == 'stevienash98765'):
        return 'stevienash98765' 
    if (username == 'plazadruben'):
        return '31wexqymjgzdkkved4vcwote7r2q' 

# users_to_add = ['stevienash98765', '31jcprt274yhbqbldnen6q4r2rxq']
display_names = ['stevienash98765', 'plazadruben', 'billywalton98765']
users_to_add = ['stevienash98765', '31wexqymjgzdkkved4vcwote7r2q', '31jcprt274yhbqbldnen6q4r2rxq']

df_playlist = addUsersToDataSet(df_playlist, users_to_add)


# df_playlist.loc[(df_playlist['user_id'] == users_to_add[0]) | (df_playlist['user_id'] == users_to_add[1])]

In [8]:
# df_playlist.loc[(df_playlist['user_id'] == users_to_add[0]) | (df_playlist['user_id'] == users_to_add[1])]
df_playlist.loc[df_playlist['user_id'] == display_names[1]]

,user_id,artist,track,playlist
12891780,plazadruben,Taylor Swift,Lavender Haze,testplaylist2
12891781,plazadruben,Taylor Swift,All Of The Girls You Loved Before,testplaylist2
12891782,plazadruben,Taylor Swift,Blank Space,testplaylist2
12891783,plazadruben,Taylor Swift,Midnight Rain,testplaylist2
12891784,plazadruben,Taylor Swift,Love Story (Taylor’s Version),testplaylist2
...,...,...,...,...
12891875,plazadruben,Selena Gomez,Lose You To Love Me,testplaylist2
12891876,plazadruben,Selena Gomez,Let Me Get Me,testplaylist2
12891877,plazadruben,Selena Gomez,The Heart Wants What It Wants,testplaylist2
12891878,plazadruben,Selena Gomez,Hands To Myself,testplaylist2


In [9]:
# Adding test user and seeing what the output will be.
# temp_user_id = '00000000000000000000000000000000'
# temp_user_artists = [
#     'Eminem', 'Kanye West', '50 Cent', 'Drake', 'Kendrick Lamar', 'Future', 
#     'Lil Baby', 'Snoop Dogg', 'Post Malone', 'Lil Wayne', 'Kid Cudi']
# temp_track_name = 'testtrackname'
# temp_playlist_name = 'testplaylist'

# new_rows = []
# for i in range(5000):
#     temp_artist = random.choice(temp_user_artists)
#     new_row = {
#         'user_id': temp_user_id,
#         'artist': temp_artist,
#         'track': temp_track_name,
#         'playlist': temp_playlist_name
#     }
#     new_rows.append(new_row)

# temp_df = pd.DataFrame(new_rows) 
# df_playlist = pd.concat([df_playlist, temp_df], ignore_index = True)
# # df_last = df_playlist.tail(10)
# # df_last

# df_playlist.loc[df_playlist['user_id'] == temp_user_id]

In [10]:
# Keeping artists that appear with a minimum frequency. 
# Only keeping artists with high frequency than 50

In [11]:
df_playlist = df_playlist.groupby('artist').filter(lambda x : len(x) >= 50)
# df_playlist
# df_playlist.loc[df_playlist['user_id'] == temp_user_id]
df_playlist.loc[df_playlist['user_id'] == display_names[1]]

,user_id,artist,track,playlist
12891780,plazadruben,Taylor Swift,Lavender Haze,testplaylist2
12891781,plazadruben,Taylor Swift,All Of The Girls You Loved Before,testplaylist2
12891782,plazadruben,Taylor Swift,Blank Space,testplaylist2
12891783,plazadruben,Taylor Swift,Midnight Rain,testplaylist2
12891784,plazadruben,Taylor Swift,Love Story (Taylor’s Version),testplaylist2
...,...,...,...,...
12891875,plazadruben,Selena Gomez,Lose You To Love Me,testplaylist2
12891876,plazadruben,Selena Gomez,Let Me Get Me,testplaylist2
12891877,plazadruben,Selena Gomez,The Heart Wants What It Wants,testplaylist2
12891878,plazadruben,Selena Gomez,Hands To Myself,testplaylist2


In [12]:
# Keeping users with at least 10 unique artists in their playlists to lessen
# the impact of COLD START PROBLEM

In [13]:
df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') >= 10]
# df_playlist.loc[df_playlist['user_id'] == temp_user_id]
# df_playlist.loc[(df_playlist['user_id'] == users_to_add[0]) | (df_playlist['user_id'] == users_to_add[1])]
df_playlist.loc[df_playlist['user_id'] == display_names[1]]

,user_id,artist,track,playlist
12891780,plazadruben,Taylor Swift,Lavender Haze,testplaylist2
12891781,plazadruben,Taylor Swift,All Of The Girls You Loved Before,testplaylist2
12891782,plazadruben,Taylor Swift,Blank Space,testplaylist2
12891783,plazadruben,Taylor Swift,Midnight Rain,testplaylist2
12891784,plazadruben,Taylor Swift,Love Story (Taylor’s Version),testplaylist2
...,...,...,...,...
12891875,plazadruben,Selena Gomez,Lose You To Love Me,testplaylist2
12891876,plazadruben,Selena Gomez,Let Me Get Me,testplaylist2
12891877,plazadruben,Selena Gomez,The Heart Wants What It Wants,testplaylist2
12891878,plazadruben,Selena Gomez,Hands To Myself,testplaylist2


In [14]:
# Grouping by the frequency count for each users artists (# of times that an
# artist appeared in playlists created by a user)

In [15]:
size = lambda x : len(x)
df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

# df_freq.loc[df_freq['user_id'] == temp_user_id]
# df_playlist.loc[(df_playlist['user_id'] == users_to_add[0]) | (df_playlist['user_id'] == users_to_add[1])]
# df_playlist.loc[df_playlist['user_id'] == display_names[2]]
df_playlist.loc[df_playlist['user_id'] == display_names[1]]

,user_id,artist,track,playlist
12891780,plazadruben,Taylor Swift,Lavender Haze,testplaylist2
12891781,plazadruben,Taylor Swift,All Of The Girls You Loved Before,testplaylist2
12891782,plazadruben,Taylor Swift,Blank Space,testplaylist2
12891783,plazadruben,Taylor Swift,Midnight Rain,testplaylist2
12891784,plazadruben,Taylor Swift,Love Story (Taylor’s Version),testplaylist2
...,...,...,...,...
12891875,plazadruben,Selena Gomez,Lose You To Love Me,testplaylist2
12891876,plazadruben,Selena Gomez,Let Me Get Me,testplaylist2
12891877,plazadruben,Selena Gomez,The Heart Wants What It Wants,testplaylist2
12891878,plazadruben,Selena Gomez,Hands To Myself,testplaylist2


In [16]:
# Creating a data frame for artists and artist id
# Artist to artist id mapping

In [17]:
df_artist = pd.DataFrame(df_freq['artist'].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Vitamin String Quartet
1,1,Lata Mangeshkar
2,2,Ilaiyaraaja
3,3,Peggy Lee
4,4,Wolfgang Amadeus Mozart


In [18]:
df_artist.shape

(23515, 2)

In [19]:
# Adding artist_id to the frequence data frame
df_freq = pd.merge(df_freq, df_artist, how='inner', on='artist')

In [20]:
#########
# Using the lightfm library to run a traditional MF (matrix factorization) model
# since the dataset doesn't include any user or artist features
# the library allows builing of a hybrid model as well. (CF AND CBF?)
# examples: https://github.com/lyst/lightfm/blob/master/examples/

In [21]:
## Parameter Tuning
def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "max_sampled": np.random.randint(5, 15),
            "num_epochs": np.random.randint(5, 50),
        }

In [22]:
# Don't know what the
# Creating an interaction matrix df from transactional type interactions
def create_interaction_matrix(df, user_col, item_col, rating_col, norm=False, threshold=None):
    interactions = df.groupby([user_col, item_col])[rating_col].sum().unstack().reset_index().fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [23]:
# Function to create a user dictionary based on their index and number in
# interaction dataset
def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [24]:
# Function to create an item dictionary based on their item_id and item name
def create_item_dict(df, id_col, name_col):
    item_dict = {}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i, name_col]
    return item_dict

In [25]:
# Runs the matrix factorization algorithm and returns the trained model
# Original runMF
# def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30, n_jobs = 4):
# #     x = sparse.csr_matrix(interactions.values)
#     model = LightFM(no_components = n_components, loss=loss, k=k)
#     model.fit(interactions, epochs=epoch, num_threads = n_jobs)
#     return model

def runMF(interactions, n_components, loss, ls, learning_rate, item_alpha, user_alpha, max_sampled, k, epoch, n_jobs = 4):
#     x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components=n_components, loss=loss, learning_schedule=ls, learning_rate=learning_rate, item_alpha=item_alpha, user_alpha=user_alpha, max_sampled=max_sampled, k=k)
    model.fit(interactions, epochs=epoch, num_threads = n_jobs)
    return model

In [26]:
# Parameter Tuning
def random_search(train, test, num_samples=500, num_threads=4):
    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")
        
        model = LightFM(**hyperparams)
        model.fit(train, epochs=num_epochs, num_threads=num_threads)
        
        test_auc = auc_score(model, test, train_interactions=train, num_threads=num_threads).mean()
        train_auc = auc_score(model, train, num_threads=num_threads).mean()
        print(f'{test_auc} {train_auc}')
        
        hyperparams["num_epochs"] = num_epochs
        
        yield (test_auc, train_auc, hyperparams, model)

In [27]:
# Produces user recommendations, prints the list of items the user already 
# prefers, prints the list of N recommended items which user will hopefully 
# prefer.
def sample_recommendation_user(model, interactions, user_id, user_dict, item_dict, threshold = 0, nrec_items = 10, show = True):
    n_users, n_items = interactions.shape # indices
    user_x = user_dict[user_id] # getting the specific user
    scores = pd.Series(model.predict(user_x, np.arange(n_items))) # model prediction
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:]
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [28]:
# Prepping the model inputs

In [29]:
# Create interaction matrix
interactions = create_interaction_matrix(df=df_freq, user_col="user_id", item_col="artist_id", rating_col='freq', norm=False, threshold=None)
interactions.head()

artist_id,0,1,2,3,4,5,6,7,8,9,...,23505,23506,23507,23508,23509,23510,23511,23512,23513,23514
user_id,,,,,,,,,,,,,,,,,,,,,
00055176fea33f6e027cd3302289378b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0007f3dd09c91198371454c608d47f22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000b0f32b5739f052b9d40fcc5c41079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c11a16c89aa4b14b328080f5954ee,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00123e0f544dee3ab006aa7f1e5725a7,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,165.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
## NEW
# Prep the data for surprise. 
# surprise_df = df_freq.copy(deep=True)
# for index, row in surprise_df.iterrows():
#     if row['freq'] > 200:
#         surprise_df.at[index, 'freq'] = 200

In [31]:
# surprise_df.head()

In [32]:
# def get_top_n(user_id, predictions, n=10):
#     """Return the top-N recommendation for each user from a set of predictions.

#     Args:
#         predictions(list of Prediction objects): The list of predictions, as
#             returned by the test method of an algorithm.
#         n(int): The number of recommendation to output for each user. Default
#             is 10.

#     Returns:
#     A dict where keys are user (raw) ids and values are lists of tuples:
#         [(raw item id, rating estimation), ...] of size n.
#     """

#     # First map the predictions to each user.
#     top_n = defaultdict(list)
#     for uid, iid, true_r, est, _ in predictions:
#         top_n[user_id].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
#     for uid, user_ratings in top_n.items():
#         user_ratings.sort(key=lambda x: x[1], reverse=True)
#         top_n[uid] = user_ratings[:n]

#     return top_n

In [33]:
# reader = Reader(rating_scale=(0,200))
# data = Dataset.load_from_df(surprise_df[["user_id", "artist_id", "freq"]], reader)
# trainingSet = data.build_full_trainset()

# algo = SVD()
# algo.fit(trainingSet)

# # Than predict ratings for all pairs (u, i) that are NOT in the training set.
# testset = trainingSet.build_anti_testset()
# predictions = algo.test(testset)

In [34]:
# top_n = get_top_n(predictions, n=10)

# # Print the recommended items for each user

# for uid, user_ratings in top_n.items():
#     print(uid, [iid for (iid, _) in user_ratings])

In [35]:
# artists_dict = create_item_dict(df=df_artist, id_col="artist_id", name_col="artist")
# artists_dict

## NEW END

In [36]:
interactions.shape

(13668, 23515)

In [37]:
# temp

In [38]:
# Create user dict
user_dict = create_user_dict(interactions=interactions)
tmp_usr = user_dict[display_names[2]]
tmp_usr

10252

In [39]:
# Create item dict
artists_dict = create_item_dict(df=df_artist, id_col="artist_id", name_col="artist")

In [40]:
# Train-test split
x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)

In [41]:
# How does a matrix factorization model work?
# https://developers.google.com/machine-learning/recommendation/collaborative/matrix

In [42]:
# Train the matrix factorization model
# Can do hyper-parameter tuning for better results
# %time
# Original model
# model = runMF(interactions = train, n_components=30, loss='warp', k=15, epoch=30, n_jobs=4)

# Model with hyper tuned params
# Params: {
#     'no_components': 59, 
#     'learning_schedule': 'adagrad',
#     'loss': 'warp', 
#     'learning_rate': 0.03366888807224179, 
#     'item_alpha': 3.0678838066972434e-08, 
#     'user_alpha': 4.7933357847505985e-09, 
#     'max_sampled': 7, 
#     'num_epochs': 26
# }
model = runMF(interactions = train, n_components=30, loss='warp', ls='adagrad', learning_rate=0.03366888807224179, item_alpha=3.0678838066972434e-08, user_alpha=4.7933357847505985e-09, max_sampled=7, k=15, epoch=26, n_jobs=4)

# file = open('cf_model.pickle', 'rb')
# model = pickle.load(file)
# file.close()

In [43]:
# ## New Code
# (test_auc, train_auc, hyperparams, model) = max(random_search(train, test, num_threads=4), key=lambda x: x[0])

# print("Best Test AUC Score: {}\nTrain AUC Score: {}\nParams: {}".format(test_auc, train_auc, hyperparams))

# ## End New Code

In [44]:
# Evaluation metrics
# Compute AUC score for train set
# Original
# train_auc = auc_score(model, train, num_threads=4).mean()
# print('Train AUC: %s' % train_auc)

In [45]:
# Compute AUC score test set
# the parameter train_interactions allows you to exlude known positives in
# training set from the predictions and score calculations
# this is to avoid re-recommending the items the user has already interacted with
# Original
# test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
# print('Test AUC: %s' % test_auc)

In [46]:
# train_precision = precision_at_k(model, train, k=10).mean()
# test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

In [47]:
# print('train Precision %.2f, test Precision %.2f.' % (train_precision, test_precision))

In [48]:
random_cell = interactions.sample()
random_user_id = random_cell.index[0]

rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = display_names[0], 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Future
2- J. Cole
3- Kendrick Lamar
4- Various Artists
5- Drake
6- 50 Cent
7- Eminem
8- Kanye West
9- Snoop Dogg
10- Lil Wayne

 Recommended Items:
1- JAY Z
2- Rihanna
3- Daft Punk
4- Beyoncé
5- Justin Timberlake
6- A$AP Rocky
7- Usher
8- OutKast
9- Kid Cudi
10- Nicki Minaj


In [49]:
# print(type(test_auc))
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = display_names[1], 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = display_names[2], 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Selena Gomez
2- Selena Gomez & The Scene
3- Ariana Grande
4- Miley Cyrus
5- Ed Sheeran
6- Katy Perry
7- The Weeknd
8- Lady Gaga
9- Drake
10- Beyoncé
11- Taylor Swift

 Recommended Items:
1- Rihanna
2- Pitbull
3- David Guetta
4- Calvin Harris
5- Maroon 5
6- Avicii
7- Jason Derulo
8- Nicki Minaj
9- Ellie Goulding
10- Bruno Mars
Known Likes:
1- The All-American Rejects
2- Plain White T's
3- Kings of Leon
4- The Goo Goo Dolls
5- The Verve
6- Guns N' Roses
7- Hinder
8- The Fray
9- Nickelback
10- The Killers
11- Weezer
12- Various Artists
13- Foo Fighters
14- Snow Patrol
15- Nirvana
16- Aerosmith
17- Linkin Park
18- Green Day
19- Nine Inch Nails
20- Radiohead
21- Metallica
22- Red Hot Chili Peppers
23- Slipknot
24- Pearl Jam

 Recommended Items:
1- Coldplay
2- Muse
3- Queen
4- Eminem
5- Katy Perry
6- Michael Jackson
7- Daft Punk
8- Kings Of Leon
9- Maroon 5
10- U2


In [50]:
## Trying to export model


with open('cf_model.pickle', 'wb') as file:
    pickle.dump(model, file)

## END export

In [51]:
## Load model get prediction

file = open('cf_model.pickle', 'rb')
pickled_model = pickle.load(file)
file.close()

rec_list = sample_recommendation_user(model = pickled_model, 
                                      interactions = interactions, 
                                      user_id = display_names[0], 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

rec_list = sample_recommendation_user(model = pickled_model, 
                                      interactions = interactions, 
                                      user_id = display_names[1], 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = display_names[2], 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Future
2- J. Cole
3- Kendrick Lamar
4- Various Artists
5- Drake
6- 50 Cent
7- Eminem
8- Kanye West
9- Snoop Dogg
10- Lil Wayne

 Recommended Items:
1- JAY Z
2- Rihanna
3- Daft Punk
4- Beyoncé
5- Justin Timberlake
6- A$AP Rocky
7- Usher
8- OutKast
9- Kid Cudi
10- Nicki Minaj
Known Likes:
1- Selena Gomez
2- Selena Gomez & The Scene
3- Ariana Grande
4- Miley Cyrus
5- Ed Sheeran
6- Katy Perry
7- The Weeknd
8- Lady Gaga
9- Drake
10- Beyoncé
11- Taylor Swift

 Recommended Items:
1- Rihanna
2- Pitbull
3- David Guetta
4- Calvin Harris
5- Maroon 5
6- Avicii
7- Jason Derulo
8- Nicki Minaj
9- Ellie Goulding
10- Bruno Mars
Known Likes:
1- The All-American Rejects
2- Plain White T's
3- Kings of Leon
4- The Goo Goo Dolls
5- The Verve
6- Guns N' Roses
7- Hinder
8- The Fray
9- Nickelback
10- The Killers
11- Weezer
12- Various Artists
13- Foo Fighters
14- Snow Patrol
15- Nirvana
16- Aerosmith
17- Linkin Park
18- Green Day
19- Nine Inch Nails
20- Radiohead
21- Metallica
22- Red Hot Chili